In [29]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

import torch 
import torch.nn.functional as F
import torch.autograd as autograd
import torch.nn as nn
from sklearn.model_selection import train_test_split



In [31]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [32]:
def PrePadding(data, seglength):
    pad = (-len(data)) % seglength
    l = np.concatenate((data,np.zeros((pad, len(data[0])))))
    return l

In [33]:
def split_to_epochs(data, seglength):
    data = PrePadding(data, seglength)
    data = np.array(np.split(data, seglength)).reshape(-1, seglength, data.shape[-1])
    return data

In [34]:
df1 = deepcopy(pd.read_csv(r"D:\Clean Data\Focal\ALAHYARI\LB.csv", skiprows=2, header=None))
df2 = deepcopy(pd.read_csv(r"D:\Clean Data\IGE\bagherimaede\LB.csv", skiprows=2, header=None))
rawdata1 = np.array(df1)
rawdata2 = np.array(df2)

In [35]:
seglength = 1500
data1 = torch.from_numpy(split_to_epochs(rawdata1, seglength))
data2 = torch.from_numpy(split_to_epochs(rawdata2, seglength))
print(f"data1 shape: {np.shape(data1)}")
print(f"data2 shape: {np.shape(data2)}")

data1 shape: torch.Size([1219, 1500, 20])
data2 shape: torch.Size([1377, 1500, 20])


In [36]:
label1 = torch.ones((data1.shape[0], 1))
label2 = torch.ones((data2.shape[0], 1)) + 1
labels = torch.cat((label1, label2))
data = torch.cat((data1, data2))
data.shape

torch.Size([2596, 1500, 20])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [43]:
class LSTM(nn.Module):
    def __init__(self, batch_size, segment_size, channel_count, num_classes, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.N = batch_size
        self.T = segment_size
        self.C = channel_count
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.bn = nn.BatchNorm2d(hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):

        lstm_out, (h, c) = self.lstm(x, None)
        x = self.bn(lstm_out)
        logits = self.fc(lstm_out[:, -1, :])
        probas = F.softmax(logits, dim=1)
        return probas

In [41]:
model = LSTM(num_classes=2, input_size=1500 * 20, hidden_size=4, num_layers=2)